# Лаба 9. Ваш собственный алгоритм рекомендаций фильмов

###  Построить рекомендательный алгоритм наилучшей предсказательной точности по метрике RMSE.

In [1]:
!hadoop fs -ls /labs/lab09data

Found 5 items
-rw-r--r--   2 hdfs hdfs     561456 2020-09-30 12:23 /labs/lab09data/links.csv
-rw-r--r--   2 hdfs hdfs    1390073 2020-09-30 12:23 /labs/lab09data/movies.csv
-rw-r--r--   2 hdfs hdfs   11364859 2020-09-30 12:23 /labs/lab09data/tags.csv
-rw-r--r--   2 hdfs hdfs  169754328 2020-09-30 12:23 /labs/lab09data/test.csv
-rw-r--r--   2 hdfs hdfs  169755196 2020-09-30 12:23 /labs/lab09data/train.csv


In [24]:
!hadoop fs -cat /labs/lab09data/train.csv | head -n10

userId,movieId,rating
1,2244,4.5
1,2464,4.5
1,6361,4.5
1,10620,4.5
1,12012,4.0
1,12879,4.5
1,14373,4.0
1,17487,4.5
1,18350,4.5
cat: Unable to write to output stream.


## Установка библиотек

In [11]:
import sys
!{sys.executable} -m pip install scikit-surprise --user

    100% |████████████████████████████████| 11.8MB 1.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for scikit-surprise ... done
  Stored in directory: /data/home/evgeny.permyakov/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise
  The script surprise is installed in '/data/home/evgeny.permyakov/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You are using pip version 10.0.1, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [54]:
import pandas as pd
from surprise import (
    SVD,
    Reader,
    Dataset,
    accuracy
)

from surprise.model_selection import train_test_split

# Грузим в датафреймы

In [13]:
df_train = pd.read_csv('train.csv')

In [37]:
df_train.rating.max()

5.0

In [41]:
df_test = pd.read_csv('test.csv')

In [42]:
df_test

,userId,movieId,rating
0,1,1414,0.0
1,1,2346,0.0
2,1,5278,0.0
3,1,9303,0.0
4,1,11817,0.0
...,...,...,...
10531559,229060,27122,0.0
10531560,229060,27170,0.0
10531561,229060,27202,0.0
10531562,229060,27233,0.0


# Обучаем SVD

https://surprise.readthedocs.io/en/stable/getting_started.html#load-from-file-example

In [38]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df_train, reader)

In [39]:
trainset = data.build_full_trainset()

In [40]:
algo = SVD()

algo.fit(trainset)

# Применяем

In [43]:
test_data = Dataset.load_from_df(df_test, reader)

In [47]:
NA, test = train_test_split(test_data, test_size = 1.0)

In [48]:
predictions = algo.test(test)

In [61]:
predictions

153686

# Результат

In [62]:
df_pred = pd.DataFrame(predictions)

In [64]:
df_pred = df_pred[['uid', 'iid', 'est']]

In [67]:
df_pred = df_pred.sort_values(by=['uid','iid'], ascending=(True,True))

In [79]:
df_pred = df_pred.rename(columns = {
    'uid':'userId',
    'iid':'movieId',
    'est':'rating'
})

In [80]:
df_pred.to_csv('../lab09.csv', index=False)